In [152]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import pytz
from datetime import datetime
import numpy as np
from IPython.display import display, clear_output, Markdown


date = datetime(2018, 11, 1, 0, 0, 0, tzinfo=pytz.utc)
#places = ['Bottrop, DE']
places = ['Bottrop, DE', 'Vantaa, FI', 'Bologna, IT', 'Hasselt, BE', 'Surrey, UK']

devices_api = 'https://api.smartcitizen.me/v0/devices/'

In [153]:
def devices(place):

    geolocator = Nominatim(user_agent="iSCAPE")

    location = geolocator.geocode(place)

    location_str = str(location.latitude) + ',' + str(location.longitude)

    params = {'near': location_str, 'within': '10000'}

    req = requests.get(devices_api, params=params).json()
    
    # for item in req:
    #     print item
    devices = pd.DataFrame(req)
    devices['iscape_site'] = place
    devices['added_at'] = pd.to_datetime(devices['added_at'])
    devices['last_reading_at'] = pd.to_datetime(devices['last_reading_at'])
    devices = devices[devices['last_reading_at'] > date.isoformat()]
    devices = devices[devices['added_at'] > date.isoformat()]

    # Probably this could be done via list comprehension but it's clearer this way
    # The problem comes with the dict within the pandas series, that it's not directly indexable from pandas (same sintax)
    list_kits = list()
    for item in devices.kit:
        if item['id'] == 20: list_kits = devices[devices.kit == item].index
    
    # Filter devices
    devices = devices.loc[list_kits]
    
    devices['total_readings'] = devices.apply(lambda row: readings(row), axis=1)
    
    devices['total_readings_days'] = devices.apply(lambda row: row['total_readings'] / (60.0*24.0), axis = 1)
    devices['total_days_last_reading'] = devices.apply(lambda row: max(0, (row['last_reading_at'] - row['added_at']).days), axis=1)
    devices['total_days_since_registered'] = devices.apply(lambda row: (datetime.now() - row['added_at']).days, axis=1)

    devices['user_name'] = devices.apply(lambda row: row['owner']['username'], axis = 1)
    
    devices['ratio_readings_days'] = devices.apply(lambda row: ratio_days(row), axis = 1)
    devices['id'] = devices['id'].astype(str)
    return devices

def ratio_days(row):
    return row['total_readings_days'] / row['total_days_since_registered'] if row['total_days_since_registered'] else np.nan

def readings(row):

    params = {'from': date.isoformat(), 'to': datetime.now().isoformat(), 'rollup': '60s', 'sensor_id': 10 }

    req = requests.get(devices_api + str(row['id']) + '/readings', params=params).json()

    readings = req['readings']

    total_readings = len(readings)

    return total_readings

In [ ]:
sites = []
for place in places:
    
    sites.append(devices(place))
    
sites = pd.concat(sites)
display(sites)   

## Charts report for all sites

In [ ]:
import matplotlib.pyplot as plot
% matplotlib inline

gg = [x for _, x in sites.groupby('iscape_site')]

for i in gg:
    i = i[['id', 'iscape_site', 'ratio_readings_days', 'total_readings', 'total_readings_days', 'total_days_last_reading', 'total_days_since_registered']]
    list_sites =  i['iscape_site'].values
    fig, axes = plot.subplots(1, 3, figsize = (16,4))
    axes[0].bar(i['id'], i ['total_readings'])
    axes[0].set_ylabel('total_readings')
    axes[0].grid(True)

    axes[1].bar(i['id'], i['total_readings_days'])
    axes[1].set_ylabel('total_readings_days')
    axes[1].grid(True)

    axes[2].bar(i['id'], i['ratio_readings_days'])
    axes[2].set_ylabel('ratio_readings_days')
    axes[2].set_ylim([0, 1])
    axes[2].grid(True)
    fig.suptitle(list_sites[0] + ' Metrics')
    

## Table report for all sites

In [ ]:
clean_sites = sites.drop(['data', 'description','added_at','last_reading_at','kit', 'total_days_since_registered', 'total_days_last_reading','mac_address', 'owner', 'state', 'system_tags', 'user_tags', 'uuid', 'updated_at' ], axis=1)  
    
clean_sites = clean_sites.sort_values(['iscape_site', 'total_readings'], ascending=[True, False])

clean_sites.index = clean_sites['id']

clean_sites = clean_sites[['iscape_site', 'name', 'user_name', 'ratio_readings_days', 'total_readings']]

clean_sites_print = clean_sites.copy()

clean_sites_print['ratio_readings_days'] = pd.Series(["{0:.2f}%".format(val * 100) for val in clean_sites_print['ratio_readings_days']], index = clean_sites.index)    

display(clean_sites_print)   

In [ ]:
## Table report per site 

In [ ]:
gg = [x for _, x in sites.groupby('iscape_site')]

for i in gg:
    
    clean_sites = i.drop(['data', 'description','added_at','last_reading_at','kit', 'total_days_last_reading','mac_address', 'owner', 'state', 'system_tags', 'user_tags', 'uuid', 'updated_at' ], axis=1)  
    
    clean_sites = clean_sites.sort_values(['iscape_site', 'total_readings'], ascending=[True, False])

    clean_sites.index = clean_sites['id']

    clean_sites = clean_sites[['iscape_site', 'name', 'user_name', 'total_days_since_registered', 'total_readings', 'ratio_readings_days']]
    
    clean_sites['ratio_readings_days'] = clean_sites.apply(lambda row: row['ratio_readings_days'] if row['ratio_readings_days'] <= 1 else 1, axis=1)
    
    clean_sites_print = clean_sites.copy()
    
    clean_sites_print['ratio_readings_days'] = pd.Series(["{0:.2f}%".format(val * 100) for val in clean_sites_print['ratio_readings_days']], index = clean_sites.index)    

    ## This is temporary for Markdown output...
    
    print '## ' + clean_sites['iscape_site'][0]
    print '\n'
    
    clean_sites_print = clean_sites_print.drop(['iscape_site'], axis=1)  
    
    print tabulate(clean_sites_print, headers="keys", tablefmt="html")
    
    print '\n'
    print '\n'

In [ ]:
## Aggregated report for all sites

In [ ]:
from tabulate import tabulate

clean_sites = sites.drop(['data', 'description','added_at','last_reading_at','kit', 'total_days_last_reading','mac_address', 'owner', 'state', 'system_tags', 'user_tags', 'uuid', 'updated_at' ], axis=1)  
    
clean_sites = clean_sites.sort_values(['iscape_site', 'total_readings'], ascending=[True, False])

clean_sites.index = clean_sites['id']

clean_sites = clean_sites[['iscape_site', 'name', 'user_name', 'total_days_since_registered', 'total_readings', 'ratio_readings_days']]
    
clean_sites['ratio_readings_days'] = clean_sites.apply(lambda row: row['ratio_readings_days'] if row['ratio_readings_days'] <= 1 else 1, axis=1)
    
clean_sites_print = clean_sites.copy()
    
clean_sites_print['ratio_readings_days'] = pd.Series(["{0:.2f}%".format(val * 100) for val in clean_sites_print['ratio_readings_days']], index = clean_sites.index)    

sites_total_records2 = clean_sites.groupby(['iscape_site'])['total_readings'].sum()

sites_total_records3 = clean_sites.groupby(['iscape_site'])['ratio_readings_days'].mean()

sites_total_records4 = clean_sites.groupby(['iscape_site'])['total_days_since_registered'].mean()

sites_total_records2.plot.bar(x='Living Labs', y='Readings')

sites_total_records5 = pd.concat([sites_total_records4, sites_total_records2, sites_total_records3], axis=1)

sites_total_records5['total_days_since_registered'] = sites_total_records5['total_days_since_registered'].astype(int)

sites_total_records5['ratio_readings_days'] = pd.Series(["{0:.0f}%".format(val * 100) for val in sites_total_records5['ratio_readings_days']], index = sites_total_records5.index)    

sites_total_records5 = sites_total_records5.sort_values(['total_readings'], ascending=[False])

display(sites_total_records5)

print tabulate(sites_total_records5, ['Sites', 'Average Days registered', 'Total Sensor Readings', 'Average'], tablefmt="html")

